In [ ]:
### please specify your input path here
INPUT_TEST_FILE = "YOUR_TEST_FILE" # csv file list locations / paths to test cases (dicom)
OUTPUT_FOLDER = "output_folder" # folder to store the output cam files in npy format

In [1]:
import os
import sys
sys.path = [
    './covn3d-same',
    './timm20221011/pytorch-image-models-master',
    './smp20210127/segmentation_models.pytorch-master/segmentation_models.pytorch-master',
    './smp20210127/pretrained-models.pytorch-master/pretrained-models.pytorch-master',
    './smp20210127/EfficientNet-PyTorch-master/EfficientNet-PyTorch-master',
] + sys.path

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

DEBUG = False
# !pip -q install ./pylibjpeg140py3/pylibjpeg-1.4.0-py3-none-any.whl
# !pip -q install ./pylibjpeg140py3/python_gdcm-3.0.17.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gc
import ast
import cv2
import time
import timm
import timm4smp
import pickle
import random
import pydicom
import argparse
import warnings
import threading
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from glob import glob
import albumentations
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from pylab import rcParams

%matplotlib inline
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device('cpu')
    print('Using CPU')

timm.__version__, timm4smp.__version__

In [6]:
image_size_seg = (128, 128, 128)
msk_size = image_size_seg[0]
image_size_cls = 512  # 这里需要固定 512
n_slice_per_c = 15
n_ch = 5

batch_size_seg = 1
num_workers = 2

In [ ]:
if DEBUG:
    df = pd.read_csv(INPUT_TEST_FILE)
else:
    df = pd.read_csv(INPUT_TEST_FILE).iloc[:1]

df.tail()

# Dataset

In [8]:
from datasets.segDataset import SegTestDataset
dataset_seg = SegTestDataset(df)
loader_seg = torch.utils.data.DataLoader(dataset_seg, batch_size=batch_size_seg, shuffle=False, num_workers=num_workers)

# Model

In [10]:
from timm4smp.models.layers.conv2d_same import Conv2dSame
from conv3d_same import Conv3dSame

def convert_3d(module):

    module_output = module
    if isinstance(module, torch.nn.BatchNorm2d):
        module_output = torch.nn.BatchNorm3d(
            module.num_features,
            module.eps,
            module.momentum,
            module.affine,
            module.track_running_stats,
        )
        if module.affine:
            with torch.no_grad():
                module_output.weight = module.weight
                module_output.bias = module.bias
        module_output.running_mean = module.running_mean
        module_output.running_var = module.running_var
        module_output.num_batches_tracked = module.num_batches_tracked
        if hasattr(module, "qconfig"):
            module_output.qconfig = module.qconfig
            
    elif isinstance(module, Conv2dSame):
        module_output = Conv3dSame(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
#             padding_mode=module.padding_mode
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.Conv2d):
        module_output = torch.nn.Conv3d(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
            padding_mode=module.padding_mode
        )
#         print(module.padding, module_output.padding)
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.MaxPool2d):
        module_output = torch.nn.MaxPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            ceil_mode=module.ceil_mode,
        )
    elif isinstance(module, torch.nn.AvgPool2d):
        module_output = torch.nn.AvgPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            ceil_mode=module.ceil_mode,
        )

    for name, child in module.named_children():
        module_output.add_module(
            name, convert_3d(child)
        )
    del module

    return module_output



class TimmSegModel(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=False):
        super(TimmSegModel, self).__init__()

        self.encoder = timm4smp.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            pretrained=pretrained
        )
        g = self.encoder(torch.rand(1, 3, 64, 64))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.unet.decoder.UnetDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )
        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
#         print(x.shape)
        global_features = [0] + self.encoder(x)[:n_blocks]
#         for f in global_features[1:]:
#             print(f.shape)
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
class TimmModel(nn.Module):
    def __init__(self, backbone, image_size, pretrained=False):
        super(TimmModel, self).__init__()
        self.image_size = image_size
        self.encoder = timm.create_model(
            backbone,
            in_chans=in_chans,
            num_classes=1,
            features_only=False,
            drop_rate=0,
            drop_path_rate=0,
            pretrained=pretrained
        )

        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone or 'nfnet' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()

        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=0, bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 1),
        )
        self.lstm2 = nn.LSTM(hdim, 256, num_layers=2, dropout=0, bidirectional=True, batch_first=True)
        self.head2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 1),
        )


    def forward(self, x):  # (bs, nc*7, ch, sz, sz)
        bs = x.shape[0]
        x = x.view(bs * n_slice_per_c * 7, in_chans, self.image_size, self.image_size)
        feat = self.encoder(x)
        feat = feat.view(bs, n_slice_per_c * 7, -1)
        feat2, _ = self.lstm2(feat)
        out = self.head2(feat2[:, 0])

        return out

# Load Models

In [11]:
models_seg = []

kernel_type = 'timm3d_v2s_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_mixup1_lr1e3_20x50ep'
backbone = 'tf_efficientnetv2_s_in21ft1k'
model_dir_seg = './seg-v2s-0911/'
n_blocks = 4
for fold in range(5):
    model = TimmSegModel(backbone, pretrained=False)
    model = convert_3d(model)
    model = model.to(device)
    load_model_file = os.path.join(model_dir_seg, f'{kernel_type}_fold{fold}_best.pth')
    sd = torch.load(load_model_file, map_location=device)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    models_seg.append(model)


kernel_type = 'timm3d_res18d_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_bs4_lr3e4_20x50ep'
backbone = 'resnet18d'
model_dir_seg = './segres18d0920/'
n_blocks = 4
for fold in range(5):
    model = TimmSegModel(backbone, pretrained=False)
    model = convert_3d(model)
    model = model.to(device)
    load_model_file = os.path.join(model_dir_seg, f'{kernel_type}_fold{fold}_best.pth')
    sd = torch.load(load_model_file, map_location=device)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    models_seg.append(model)

len(models_seg)

10

In [12]:
kernel_type = '0920_2d_lstmv22headv2_convnn_512_15_6ch_lossv3_augv2_mixupv3p5_dpr3_drl3_rov1p2_rov3p2_bs4_lr6e6_eta6e6_lw151_50ep_ddp'
model_dir_cls = './cls-convnn512-1011'
backbone = 'convnext_nano'
in_chans = 6
models_cls = []

for fold in range(5):
    model = TimmModel(backbone, image_size=512, pretrained=False)
    model = model.to(device)
    load_model_file = os.path.join(model_dir_cls, f'{kernel_type}_fold{fold}_best.pth')
    sd = torch.load(load_model_file, map_location=device)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    models_cls.append(model)
    

kernel_type = '0920_2d_lstmv22headv2_convnpc_512_15_6ch_lossv3_augv2_mixupv3p5_dpr3_drl3_rov1p2_rov3p2_bs4_lr6e6_eta6e6_lw151_50ep_ddp'
model_dir_cls = './clsconvnpc5121023'
backbone = 'convnext_pico_ols'
for fold in [3, 4]:
    model = TimmModel(backbone, image_size=512, pretrained=False)
    model = model.to(device)
    load_model_file = os.path.join(model_dir_cls, f'{kernel_type}_fold{fold}_best.pth')
    sd = torch.load(load_model_file, map_location=device)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    models_cls.append(model)


kernel_type = '0920_2d_lstmv22headv2_convnt_384_15_6ch_lossv3_augv2_mixupv3p5_dpr3_drl3_rov1p2_rov3p2_bs4_lr10e6_eta10e6_lw151_50ep_ddp'
model_dir_cls = './clsconvnt1019'
backbone = 'convnext_tiny_in22ft1k'
for fold in [5, 6]:
    model = TimmModel(backbone, image_size=384, pretrained=False)
    model = model.to(device)
    load_model_file = os.path.join(model_dir_cls, f'{kernel_type}_fold{fold}_best.pth')
    sd = torch.load(load_model_file, map_location=device)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    models_cls.append(model)
    
    
kernel_type = '0920_2d_lstmv22headv2_nfl0_384_15_6ch_lossv3_augv2_mixupv3p5_dpr3_drl3_rov1p2_rov3p2_bs4_lr15e6_eta15e6_lw151_50ep_ddp'
model_dir_cls = './clsnfl03841026'
backbone = 'eca_nfnet_l0'
for fold in [1, 9]:
    model = TimmModel(backbone, image_size=384, pretrained=False)
    model = model.to(device)
    load_model_file = os.path.join(model_dir_cls, f'{kernel_type}_fold{fold}_best.pth')
    sd = torch.load(load_model_file, map_location=device)
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    models_cls.append(model)


len(models_cls)

11

In [14]:
def load_bone(msk, cid, t_paths, cropped_images, crop_info):
    n_scans = len(t_paths)
    bone = []
    try:
        msk_b = msk[cid] > 0.2
        msk_c = msk[cid] > 0.05

        x = np.where(msk_b.sum(1).sum(1) > 0)[0]
        y = np.where(msk_b.sum(0).sum(1) > 0)[0]
        z = np.where(msk_b.sum(0).sum(0) > 0)[0]

        if len(x) == 0 or len(y) == 0 or len(z) == 0:
            x = np.where(msk_c.sum(1).sum(1) > 0)[0]
            y = np.where(msk_c.sum(0).sum(1) > 0)[0]
            z = np.where(msk_c.sum(0).sum(0) > 0)[0]

        x1, x2 = max(0, x[0] - 1), min(msk.shape[1], x[-1] + 1)
        y1, y2 = max(0, y[0] - 1), min(msk.shape[2], y[-1] + 1)
        z1, z2 = max(0, z[0] - 1), min(msk.shape[3], z[-1] + 1)
        zz1, zz2 = int(z1 / msk_size * n_scans), int(z2 / msk_size * n_scans)


        inds = np.linspace(zz1 ,zz2-1 ,n_slice_per_c).astype(int)
        inds_ = np.linspace(z1 ,z2-1 ,n_slice_per_c).astype(int)
        for sid, (ind, ind_) in enumerate(zip(inds, inds_)):

            msk_this = msk[cid, :, :, ind_]

            images = []
            for i in range(-n_ch//2+1, n_ch//2+1):
                try:
                    dicom = pydicom.read_file(t_paths[ind+i])
                    images.append(dicom.pixel_array)
                except:
                    images.append(np.zeros((512, 512)))

            data = np.stack(images, -1)
            data = data - np.min(data)
            data = data / (np.max(data) + 1e-4)
            data = (data * 255).astype(np.uint8)
            msk_this = msk_this[x1:x2, y1:y2]
            xx1 = int(x1 / msk_size * data.shape[0])
            xx2 = int(x2 / msk_size * data.shape[0])
            yy1 = int(y1 / msk_size * data.shape[1])
            yy2 = int(y2 / msk_size * data.shape[1])
            data = data[xx1:xx2, yy1:yy2]
            data = np.stack([cv2.resize(data[:, :, i], (image_size_cls, image_size_cls), interpolation = cv2.INTER_LINEAR) for i in range(n_ch)], -1)
            msk_this = (msk_this * 255).astype(np.uint8)
            msk_this = cv2.resize(msk_this, (image_size_cls, image_size_cls), interpolation = cv2.INTER_LINEAR)

            data = np.concatenate([data, msk_this[:, :, np.newaxis]], -1)

            bone.append(torch.tensor(data))      
        crop_info[cid] = [xx1, xx2, yy1, yy2, zz1, zz2, n_scans]
             
    except:
        for sid in range(n_slice_per_c):
            bone.append(torch.ones((image_size_cls, image_size_cls, n_ch+1)).int())

    cropped_images[cid] = torch.stack(bone, 0)
    


def load_cropped_images(msk, image_folder, n_ch=n_ch):

    t_paths = sorted(glob(os.path.join(image_folder, "*")), key=lambda x: int(x.split('/')[-1].split(".")[0].split("-")[-1]))
    for cid in range(7):
        threads[cid] = threading.Thread(target=load_bone, args=(msk, cid, t_paths, cropped_images, crop_info))
        threads[cid].start()
    for cid in range(7):
        threads[cid].join()

    return torch.cat(cropped_images, 0), crop_info


def get_trans(img, I):
    I = I % 8
    if I >= 4:
        img = img.transpose(3, 4)
    if I % 4 == 0:
        return img
    elif I % 4 == 1:
        return img.flip(3)
    elif I % 4 == 2:
        return img.flip(4)
    elif I % 4 == 3:
        return img.flip(3).flip(4)
    
    
def get_cam_trans(cam, I):
    I = I % 8
    if I >= 4:
        cam = cam.transpose(1, 2)
    if I % 4 == 0:
        return cam
    elif I % 4 == 1:
        return cam.flip(1)
    elif I % 4 == 2:
        return cam.flip(2)
    elif I % 4 == 3:
        return cam.flip(1).flip(2)

# Predict

In [15]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import BinaryClassifierOutputTarget
# 0:9 encoder.stages[-1].blocks[-1]
# 9:11 encoder.final_act

sys.path.append('../custom_grad_cam/') # customized grad cam lib
from interpolate_voxels import cam_to_intermediate_cam

targets = [BinaryClassifierOutputTarget(1)] # BinaryClassifierOutputTarget

In [16]:
def get_cam(model, target_layers, input_tensor, I):
    with GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available()) as cam:
        # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
        local_cam = get_cam_trans(torch.tensor(cam(input_tensor=get_trans(input_tensor, I), targets=targets)), I)
        if local_cam.shape[-1] != 512:
            local_cam = F.interpolate(local_cam.unsqueeze(0), size=512, mode='bilinear').squeeze(0)
        local_cam = local_cam.cpu().numpy()
        
    return local_cam

In [18]:
bar = tqdm(loader_seg)
# with torch.no_grad():
#     with amp.autocast():
for batch_id, (images) in enumerate(bar):
    torch.backends.cudnn.enabled = True
    images = images.to(device)

    # SEG
    pred_masks = []
    for model in models_seg:
        pmask = model(images).sigmoid()
        pred_masks.append(pmask)
#                 print(pmask)
    pred_masks = torch.stack(pred_masks, 0).mean(0).cpu().detach().numpy()

    # Build cls input
    cls_inp = []
    threads = [None] * 7
    cropped_images = [None] * 7
    crop_info = [None] * 7
    
    name = None
    for i in range(pred_masks.shape[0]):
        row = df.iloc[batch_id*batch_size_seg+i]
        cropped_images, crop_info = load_cropped_images(pred_masks[i], row.image_folder)
        cube_id = row['StudyInstanceUID']
        cls_inp.append(cropped_images.permute(0, 3, 1, 2).float() / 255.)
    cls_inp = torch.stack(cls_inp, 0).to(device)  # (1, 105, 6, 512, 512)

    torch.backends.cudnn.enabled = False
    grayscale_cam = np.zeros((105, 512, 512))
    # CLS 512
    for I, model in enumerate(models_cls[:7]):
        grayscale_cam += get_cam(model, [model.encoder.stages[-1].blocks[-1]], cls_inp, I)

    ### 512 -> 384
    cls_inp = cls_inp.view(7, 15, 6, 512, 512).contiguous()
    cls_inp = torch.stack([F.interpolate(cls_inp[i], size=384, mode='bilinear') for i in range(7)], 0)
    ###

    # CLS 384
    cls_inp = cls_inp.view(-1, 105, 6, 384, 384).contiguous()
    for I, model in enumerate(models_cls[7:9]):
        grayscale_cam += get_cam(model, [model.encoder.stages[-1].blocks[-1]], cls_inp, I+7)

    for I, model in enumerate(models_cls[9:]):
        grayscale_cam += get_cam(model, [model.encoder.final_act], cls_inp, I+9)

    grayscale_cam = grayscale_cam / 11


    n_scans = -1
    for c_info in crop_info:
        if c_info is not None:
            n_scans = c_info[-1]
            break
    if n_scans == -1:
        print(f"Cannot Process case {cube_id}, skip for now")
    
    cam_ = np.zeros((n_scans, 512, 512))
    for cid in range(7):
        if crop_info[cid] is None:
            continue
        xx1, xx2, yy1, yy2, zz1, zz2, _ = crop_info[cid]
        cam_[zz1:zz2, xx1:xx2, yy1:yy2] += cam_to_intermediate_cam(grayscale_cam[cid*15:(cid+1)*15], zz2-zz1, zz2-zz1, xx2-xx1, yy2-yy1)

    # Define the file path for saving
    save_path = os.path.join(OUTPUT_FOLDER, f"{cube_id}.npy")

    # Save the resized_cam as a .npy file
    np.save(save_path, cam_)



100%|██████████| 102/102 [33:32<00:00, 19.73s/it]
